# Steps

## 1. [Import the Libraries](#1.-Import-the-libraries)
## 2. [Load the data](#2.-Load-the-Data)
## 3. [Preprocessing](#3.-Preprocessing-the-Data)
- 3.1 [OneHotEncoding](#3.1-OneHotEncoding-with-LabelEncoder)
- 3.2 [Standard Scaling](#3.2-Standard-Scaling)
## 4. [Data preparation](#4.-Data-Preparation)
## 5. [Model training](#5.-Model-Training)
## 6. [Model Evaluation]

## 1. Import the libraries

[🔝 Return to top](#Steps)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
import warnings
warnings.filterwarnings('ignore')

## 2. Load the Data

[🔝 Return to top](#Steps)

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.set_index('id', inplace = True)
test_data.set_index('id', inplace = True)

## 3. Preprocessing the Data

[🔝 Return to top](#Steps)

In [3]:
numerical_columns = [ col for col in train_data.columns if train_data[col].dtype != 'O']
#['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
categorical_columns = [col for col in train_data.columns if train_data[col].dtype == 'object']
#['Soil Type', 'Crop Type', 'Fertilizer Name']

In [4]:
numerical_columns

['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']

In [5]:
train_data.iloc[:1]

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
id,,,,,,,,,
0,37,70,36,Clayey,Sugarcane,36,4,5,28-28


### 3.1 OneHotEncoding with LabelEncoder

[🔝 Return to top](#Steps)

In [6]:
labeler = LabelEncoder()
test_data['Fertilizer Name'] = 'z'
for col_name in categorical_columns:
    test_data[col_name] = labeler.fit_transform(test_data[col_name])
    train_data[col_name] = labeler.fit_transform(train_data[col_name])

### 3.2 Standard Scaling

[🔝 Return to top](#Steps)

## 4. Data Preparation

[🔝 Return to top](#Steps)

In [7]:
X = train_data.drop(['Fertilizer Name'], axis = 1)
y = train_data['Fertilizer Name']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## 5. Model Training 

[🔝 Return to top](#Steps)

In [8]:
model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [9]:
probs = model.predict_proba(test_data.drop(['Fertilizer Name'], axis =1))
top3_indices = np.argsort(probs , axis = 1)[ : , ::-1][:,:3]
top3_labels = labeler.inverse_transform(top3_indices.ravel()).reshape(top3_indices.shape)

pred_top3 = [' '.join(row) for row in top3_labels]

In [10]:
submission = pd.DataFrame({
    "id" : test_data.index,
    "Fertilizer Name" : pred_top3
})

submission.to_csv("submission.csv", index = False) 
submission.head()

,id,Fertilizer Name
0,750000,DAP 28-28 14-35-14
1,750001,17-17-17 20-20 10-26-26
2,750002,20-20 14-35-14 10-26-26
3,750003,14-35-14 17-17-17 10-26-26
4,750004,20-20 10-26-26 17-17-17
